In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import xarray as xr
import pandas as pd
from get_map import weather_radar_thredds

### Download map

![Cordinates for interesting area from bboxfinder.com](bbox.png)

### Plot radar image

In [ ]:
min_lat = 59.1759
max_lat = 61.4184
min_lon = 9.0527
max_lon = 12.0575

resolution = 100 # grid for plotting weather stations
frame = 0
x_grid = np.linspace(x_data.min() - frame, x_data.max() + frame, resolution)
y_grid = np.linspace(y_data.min() - frame, y_data.max() + frame, resolution)

min_lat = y_data.min() - frame
max_lat = y_data.max() + frame
min_lon = x_data.min() - frame
max_lon = x_data.max() + frame

radar_reflectivity = weather_radar_thredds(min_lat, max_lat, min_lon, max_lon)
fig, ax = plt.subplots(figsize=(20,8))
ax.imshow(radar_reflectivity) 
ax.scatter(x_data_trans, y_data_trans, c='red', marker='o')
plt.show()